# Creat a Dataset

1. We downloaded the data from <a href=''>here</a> to our local computer.
2. We extracted the zip file.
3. We Investegated the directories and file.
4. We wrote a script which will extract the relevant data into an EXCEL.
5. Then we uplaoded the EXCEL file to google drive.

<br><br><br>

### Here is the code we used to create the mentioned EXCEL file:

Imports

In [4]:
import os

import json
import pandas as pd
from tqdm import tqdm

from google.colab import drive

In [5]:
drive.mount(r'/content/drive/',force_remount=True) 

data_dir = os.getcwd() + '/drive/My Drive/Colab Notebooks/AdvancedML/HW3/'
data_dir

Mounted at /content/drive/


'/content/drive/My Drive/Colab Notebooks/AdvancedML/HW3/'

Create a class for the 'body text' as JSON

In [ ]:
class JsonFullBodyText:

    def __init__(self, file_path):
  
        with open(file_path) as file:
            content = json.load(file)
        self.body_text = []
  
        # get all body text
        for entry in content['body_text']:
            self.body_text.append(entry['text'])
  
        self.body_text = '\n'.join(self.body_text)

Load Data

In [ ]:
metadata_filename = "metadata.csv"
root_path = r"C:\IDC\third year\Advanced ML\HW\HW3\\" 
# filepath = r"C:\IDC\third year\Advanced ML\HW\HW3\\metadata.csv"

df = pd.read_csv(root_path + metadata_filename)
# print(f"Total of {df.shape[0]} articles in the dataset")

We keep the papers which contains text

In [ ]:
df.dropna(subset=["pdf_json_files"], inplace=True)
# print(f"Total of {df.shape[0]} articles after removing records without text")

We sort the article by date

In [ ]:
df.sort_values(by=["publish_time"], ascending=False, inplace=True)
df.reset_index(inplace = True)

And extract the rlevant data from the most current 10K articles

In [ ]:
num_valid_articles = 0
MAX_ARTICLES_TO_PROCESS = 10_000

final_df = pd.DataFrame(columns=["title", "cord_uid", "abstract", "body_text"])

In [ ]:
for ind, row in df.iterrows():
    temp_df = pd.DataFrame(columns=["title", "cord_uid", "abstract", "body_text"])
    json_filepath = row["pmc_json_files"] if type(row["pmc_json_files"]) == str else row["pdf_json_files"]
    if ";" in json_filepath:
        print(f"Multiple json paths for record {ind}")
        continue
    json_full_body_text = JsonFullBodyText(root_path + json_filepath)

    # get title
    title = row["title"]
    temp_df['title'] = [title]

    # get paper id
    cord_uid = row["cord_uid"]
    temp_df['cord_uid'] = [cord_uid]

    # get abstract
    temp_df['abstract'] = row["abstract"] if type(row["abstract"]) == str and len(row["abstract"]) > 0 else "no abstract"

    # get body text
    temp_df['body_text'] = [json_full_body_text.body_text]

    final_df = final_df.append(temp_df)
    num_valid_articles += 1

    print(f"Processed {num_valid_articles} / {MAX_ARTICLES_TO_PROCESS}")
    if num_valid_articles >= MAX_ARTICLES_TO_PROCESS:
        break

Save the extracted data to an EXCEL file

In [ ]:
final_df.to_excel(root_path + "full_articles_10k.xlsx", index=False)

<br><br><br>
And this is the result:

In [6]:
df = pd.read_excel(data_dir + 'full_articles_10k.xlsx')
df.head()

,title,cord_uid,abstract,body_text
0,Chapter 6 Best practices and approaches using ...,rsgdqk1x,This chapter provides ten practical case studi...,"Throughout the previous chapters, this book ad..."
1,A Comparison: Prediction of Death and Infected...,k58agggx,"COVID-19 is a virus causing pneumonia, also kn...","COVID-19 is a virus causing pneumonia, also kn..."
2,Environmental implication of personal protecti...,ercubdf9,"In the present global health emergency, face m...",Due to the fast development in the domain of c...
3,The first three weeks of lockdown in England: ...,3u446iuy,With the outbreak of COVID-19 being declared a...,The first UK confirmed death from COVID-19 was...
4,Appendix B Climate change and global warming: ...,ora47nlw,Unknown,Climate change and global warming:\nImpacts on...
